In [55]:
%gui qt
%matplotlib qt

import os
from pprint import pprint

import numpy as np
from matplotlib import pyplot as plt

import pandas as pd
from IPython.display import HTML, display

from pyqtgraph.Qt import QtGui, QtCore

In [2]:
import qcodes as qc
from qcodes import ParamSpec, new_data_set
from qcodes.dataset.data_set import DataSet
from qcodes.dataset.measurements import Measurement
from qcodes.dataset.experiment_container import (Experiment,
                                                 load_last_experiment,
                                                 new_experiment)
from qcodes.dataset.database import initialise_database
from qcodes.dataset.sqlite_base import (connect, get_experiments, get_runs,
                                        get_dependencies, get_dependents, get_layout)

In [56]:
# DB_PATH = "./test_10k_sets.db"
DB_PATH = "./experiments_BF2.db"

In [4]:
def test_data(nx, ny):
    x = np.arange(nx, dtype=np.float64)
    y = np.arange(ny, dtype=np.float64)
    xx, yy = np.meshgrid(x, y)
    zz = xx * yy
    
    return xx.reshape(-1), yy.reshape(-1), zz.reshape(-1)

def insert_data(name='dummy_data', nx=10, ny=10):
    ds = new_data_set('dummy_data', 
                      specs=[ParamSpec('x', 'array', unit='V'),
                             ParamSpec('y', 'array', unit='A'),
                             ParamSpec('z1', 'array', unit='C', depends_on=['x', 'y']),
                             ParamSpec('z2', 'array', unit='C', depends_on=['x', 'y'])])
    x, y, z = test_data(nx, ny)
    ds.add_result(dict(x=x, y=y, z1=z, z2=z))
    ds.mark_complete()
    
    return ds.run_id

def make_test_db(nruns=100, nx=100, ny=100):
    qc.config.core.db_location = DB_PATH
    initialise_database()
    
    for i in range(nruns):
        print(f'Inserting run {i+1}/{nruns}' + 10*' ', end='\r')
        if not i%25:
            new_experiment(name=f'test_exp_{i//25}', sample_name="no sample")
    
        insert_data(nx, ny)

In [5]:
# make_test_db(nruns=10000)

In [141]:
def get_ds_structure(ds):
    structure = {}

    # for each data param (non-independent param)
    for dependent_id in get_dependents(ds.conn, ds.run_id):
        
        # get name etc.
        layout = get_layout(ds.conn, dependent_id)
        name = layout['name']
        structure[name] = {'values' : [], 'unit' : layout['unit'], 'axes' : []}

        # find dependencies (i.e., axes) and add their names/units in the right order
        dependencies = get_dependencies(ds.conn, dependent_id)
        for dep_id, iax in dependencies:
            dep_layout = get_layout(ds.conn, dep_id)
            dep_name = dep_layout['name']
            structure[name]['axes'].insert(iax, dep_name)
            structure[dep_name] = {'values' : [], 'unit' : dep_layout['unit']}

    return structure

def get_ds_info(conn, run_id, get_structure=True):
    ds = DataSet(conn=conn, run_id=run_id)
    
    ret = {}
    ret['experiment'] = ds.exp_name
    ret['sample'] = ds.sample_name
    
    _complete_ts = ds.completed_timestamp()
    if _complete_ts is not None:
        ret['completed date'] = _complete_ts[:10]
        ret['completed time'] = _complete_ts[11:]
    else:
        ret['completed date'] = ''
        ret['completed time'] = ''
    
    _start_ts = ds.run_timestamp()
    ret['started date'] = _start_ts[:10]
    ret['started time'] = _start_ts[11:]
     
    if get_structure:
        ret['structure'] = get_ds_structure(ds)
    
    ret['records'] = ds.number_of_results
        
    return ret

def get_ds_info_from_path(path, run_id, get_structure=True):
    ds = DataSet(path_to_db=path, run_id=run_id)
    return get_ds_info(ds.conn, run_id, get_structure=get_structure)
    
    
def get_runs_from_db(path, start=0, stop=None, get_structure=False):    
    conn = connect(path)
    runs = get_runs(conn)
    
    if stop is None:
        stop = len(runs)
    
    runs = runs[start:stop]
    overview = {}
    
    for run in runs:
        run_id = run['run_id']
        overview[run_id] = get_ds_info(conn, run_id, get_structure=get_structure)
    
    return overview

def get_runs_from_db_as_dataframe(path, *arg, **kw):
    overview = get_runs_from_db(path, *arg, **kw)
    df = pd.DataFrame.from_dict(overview, orient='index')
    return df

In [192]:
def dictToTreeWidgetItems(d):
    items = []
    for k, v in d.items():
        if not isinstance(v, dict):
            item = QtGui.QTreeWidgetItem([str(k), str(v)])
        else:
            item = QtGui.QTreeWidgetItem([k, ''])
            for child in dictToTreeWidgetItems(v):
                item.addChild(child)
        items.append(item)
    return items

class DateList(QtGui.QListWidget):
    
    datesSelected = QtCore.pyqtSignal(list)
    
    def __init__(self, parent=None):
        super().__init__(parent)
    
        self.setSelectionMode(QtGui.QListView.ExtendedSelection)
        self.itemSelectionChanged.connect(self.sendSelectedDates)
    
    @QtCore.pyqtSlot(list)
    def updateDates(self, dates):
        for d in dates:
            if len(self.findItems(d, QtCore.Qt.MatchExactly)) == 0:
                self.insertItem(0, d)
                
        self.sortItems(QtCore.Qt.DescendingOrder)
    
    @QtCore.pyqtSlot()
    def sendSelectedDates(self):
        selection = [item.text() for item in self.selectedItems()]
        self.datesSelected.emit(selection)
    

class RunList(QtGui.QTreeWidget):
    
    cols = ['Run ID', 'Experiment', 'Sample', 'Started', 'Completed', 'Records']
    
    runSelected = QtCore.pyqtSignal(int)
    
    def __init__(self, parent=None):
        super().__init__(parent)
        
        self.setColumnCount(len(self.cols))
        self.setHeaderLabels(self.cols)
        
        self.itemSelectionChanged.connect(self.selectRun)
        self.itemActivated.connect(self.activateRun)
        
    def addRun(self, runId, **vals):
        lst = [str(runId)]
        lst.append(vals.get('experiment', ''))
        lst.append(vals.get('sample', ''))
        lst.append(vals.get('started date', '') + ' ' + vals.get('started time', ''))
        lst.append(vals.get('started date', '') + ' ' + vals.get('completed time', ''))
        lst.append(str(vals.get('records', '')))
        
        item = QtGui.QTreeWidgetItem(lst)
        self.addTopLevelItem(item)
        
    def setRuns(self, selection):
        self.clear()
        for runId, record in selection.items():
            self.addRun(runId, **record)
        
        for i in range(len(self.cols)):
            self.resizeColumnToContents(i)
    
    @QtCore.pyqtSlot()
    def selectRun(self):
        selection = self.selectedItems()
        if len(selection) == 0:
            return
        
        runId = int(selection[0].text(0))
        self.runSelected.emit(runId)
        
    @QtCore.pyqtSlot(QtGui.QTreeWidgetItem, int)
    def activateRun(self, item, column):
        runId = int(item.text(0))
        print(f'should now launch plotting of {runId}')


class RunInfo(QtGui.QTreeWidget):
    
    def __init__(self, parent=None):
        super().__init__(parent)
        
        self.setHeaderLabels(['Key', 'Value'])
        self.setColumnCount(2)
        
    @QtCore.pyqtSlot(dict)
    def setInfo(self, infoDict):
        self.clear()
        
        items = dictToTreeWidgetItems(infoDict)
        for item in items:
            self.addTopLevelItem(item)
            item.setExpanded(True)
        
        self.expandAll()
        for i in range(2):
            self.resizeColumnToContents(i)

class QCodesDBInspector(QtGui.QMainWindow):
    
    dbdfUpdated = QtCore.pyqtSignal()
    sendInfo = QtCore.pyqtSignal(dict)
    
    def __init__(self, parent=None, dbPath=None):
        super().__init__(parent)
        
        self.filepath = dbPath
        
        # Main Selection widgets
        self.dateList = DateList()
        self.runList = RunList()
        self.runInfo = RunInfo()
        
        rightSplitter = QtGui.QSplitter(QtCore.Qt.Vertical)
        rightSplitter.addWidget(self.runList)
        rightSplitter.addWidget(self.runInfo)
        rightSplitter.setSizes([400, 200])
        
        splitter = QtGui.QSplitter()
        splitter.addWidget(self.dateList)
        splitter.addWidget(rightSplitter)
        splitter.setSizes([100, 500])
        
        self.setCentralWidget(splitter)
        
        # status bar
        self.status = QtGui.QStatusBar()
        self.setStatusBar(self.status)
        
        # toolbar
        self.toolbar = self.addToolBar('Options')
        
        # sizing
        self.resize(640, 640)
        
        # connect signals/slots
        self.dbdfUpdated.connect(self.updateDates)
        self.dbdfUpdated.connect(self.showDBPath)
        
        self.dateList.datesSelected.connect(self.setDateSelection)
        self.runList.runSelected.connect(self.setRunSelection)
        self.sendInfo.connect(self.runInfo.setInfo)
        
        if self.filepath is not None:
            self.loadFullDB(self.filepath)
        
    @QtCore.pyqtSlot()
    def showDBPath(self):
        path = os.path.abspath(self.filepath)
        self.status.showMessage(path)
    
    def loadFullDB(self, path=None):
        if path is not None and path != self.filepath:
            self.filepath = path
            
        self.dbdf = get_runs_from_db_as_dataframe(self.filepath)
        self.dbdfUpdated.emit()
    
    @QtCore.pyqtSlot()
    def updateDates(self):
        dates = list(self.dbdf.groupby('started date').indices.keys())
        self.dateList.updateDates(dates)
        
    @QtCore.pyqtSlot(list)
    def setDateSelection(self, dates):
        selection = self.dbdf.loc[self.dbdf['started date'].isin(dates)].sort_index(ascending=False)
        self.runList.setRuns(selection.to_dict(orient='index'))
        
    @QtCore.pyqtSlot(int)
    def setRunSelection(self, runId):
        info = get_ds_info_from_path(self.filepath, runId, get_structure=True)
        structure = info['structure']
        for k, v in structure.items():
            v.pop('values')
        contentInfo = {'data' : structure}
        self.sendInfo.emit(contentInfo)


In [193]:
win = QCodesDBInspector(dbPath=DB_PATH)
win.show()

should now launch plotting of 109
should now launch plotting of 91


In [148]:
info = {'data': {'frequency': {'unit': 'Hz'},
          'magnitude': {'axes': ['frequency'], 'unit': 'dB'},
          'phase': {'axes': ['frequency'], 'unit': 'rad'}}}

In [154]:
def iterateDict(d, level=0):
    for k, v in d.items():
        if isinstance(v, dict):
            print(level*'', '+', k)
            iterateDict(v, level+1)
        else:
            print(level*' ', '>', k, '=', v)
            
iterateDict(info)

 + data
 + frequency
   > unit = Hz
 + magnitude
   > axes = ['frequency']
   > unit = dB
 + phase
   > axes = ['frequency']
   > unit = rad


In [84]:
dbdf = get_runs_from_db_as_dataframe(DB_PATH, start=0)
dbdf

,experiment,sample,completed date,completed time,started date,started time,records
1,testing,no sample,2018-09-26,21:33:30,2018-09-26,21:33:28,1
2,testing,no sample,2018-09-26,22:17:28,2018-09-26,22:17:27,0
3,testing,no sample,2018-09-26,22:17:41,2018-09-26,22:17:39,1
4,testing,no sample,2018-09-27,16:54:44,2018-09-27,16:54:43,0
5,testing,no sample,2018-09-27,16:55:28,2018-09-27,16:55:27,0
6,testing,no sample,2018-09-27,16:55:57,2018-09-27,16:55:55,0
7,testing,no sample,2018-09-27,16:56:33,2018-09-27,16:56:31,0
8,testing,no sample,2018-09-27,16:57:33,2018-09-27,16:57:32,0
9,testing,no sample,2018-09-27,16:58:22,2018-09-27,16:58:21,0
10,testing,no sample,2018-09-27,16:59:36,2018-09-27,16:59:34,1


In [96]:
selection = dbdf.loc[dbdf['started date'].isin(['2018-10-26', ])]

In [109]:
selection.sort_index(ascending=False)

,experiment,sample,completed date,completed time,started date,started time,records
127,VNA,FluxoniumTest_CD20181015,2018-10-26,16:58:53,2018-10-26,16:58:37,1
126,VNA,FluxoniumTest_CD20181015,2018-10-26,16:42:36,2018-10-26,16:42:21,1
